Problem: https://www.hackerearth.com/problem/machine-learning/pet-adoption-9-5838c75b/

In [343]:
import numpy as np
import pandas as pd
import os

In [344]:
train=pd.read_csv("../input/hackerearth-ml-challenge-pet-adoption/train.csv")
test=pd.read_csv("../input/hackerearth-ml-challenge-pet-adoption/test.csv")

In [345]:
train.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [346]:
test.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [347]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pet_id          18834 non-null  object 
 1   issue_date      18834 non-null  object 
 2   listing_date    18834 non-null  object 
 3   condition       17357 non-null  float64
 4   color_type      18834 non-null  object 
 5   length(m)       18834 non-null  float64
 6   height(cm)      18834 non-null  float64
 7   X1              18834 non-null  int64  
 8   X2              18834 non-null  int64  
 9   breed_category  18834 non-null  float64
 10  pet_category    18834 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 1.6+ MB


In [348]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8072 entries, 0 to 8071
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pet_id        8072 non-null   object 
 1   issue_date    8072 non-null   object 
 2   listing_date  8072 non-null   object 
 3   condition     7453 non-null   float64
 4   color_type    8072 non-null   object 
 5   length(m)     8072 non-null   float64
 6   height(cm)    8072 non-null   float64
 7   X1            8072 non-null   int64  
 8   X2            8072 non-null   int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 567.7+ KB


We have some null values in the **condition** feature column.

In [349]:
train['breed_category'].value_counts()

0.0    9000
1.0    8357
2.0    1477
Name: breed_category, dtype: int64

In [350]:
train['breed_category'][(np.isnan(train['condition']))].value_counts()

2.0    1477
Name: breed_category, dtype: int64

In [351]:
test_id = test['pet_id']

We will first make model for gettiong output y1. Then we will use y1 as input in 2nd Model to get y2.

In [352]:
y1 = train['breed_category']
y2 = train['pet_category']

Combine train and test dataset..

In [353]:
combine = pd.concat([train, test]).reset_index(drop=True)
combine.drop(['breed_category', 'pet_category'], axis=1, inplace=True)

Fill missing values with most frequent values.

In [354]:
combine['condition'].value_counts()

1.0    9747
0.0    8966
2.0    6097
Name: condition, dtype: int64

In [355]:
combine['condition'].fillna(combine['condition'].value_counts().index[0], inplace=True)

In [356]:
combine['condition'].value_counts()

1.0    11843
0.0     8966
2.0     6097
Name: condition, dtype: int64

In [357]:
combine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26906 entries, 0 to 26905
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pet_id        26906 non-null  object 
 1   issue_date    26906 non-null  object 
 2   listing_date  26906 non-null  object 
 3   condition     26906 non-null  float64
 4   color_type    26906 non-null  object 
 5   length(m)     26906 non-null  float64
 6   height(cm)    26906 non-null  float64
 7   X1            26906 non-null  int64  
 8   X2            26906 non-null  int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 1.8+ MB


## Feature Engineering
### Get months of Birth

In [358]:
combine['issue_date'] = pd.to_datetime(combine['issue_date'])
combine['listing_date'] = pd.to_datetime(combine['listing_date'])

In [359]:
issue, listing = [], []
for i in combine['issue_date']:
    issue.append(i.month)
for j in combine['listing_date']:
    listing.append(j.month)

combine['issue_month'] = issue
combine['listing_month'] = listing

### Find the days to become matured.

In [360]:
combine['days'] =  (combine['listing_date'] - combine['issue_date']).dt.days

### Take some parts from ID as they indicate different breeds/ species.

In [361]:
combine['serial'] = combine['pet_id'].str[:6]
combine['serial2'] = combine['pet_id'].str[:7]

In [363]:
combine['serial'].head(3)

0    ANSL_6
1    ANSL_6
2    ANSL_6
Name: serial, dtype: object

In [364]:
combine['serial2'].head(3)

0    ANSL_69
1    ANSL_66
2    ANSL_69
Name: serial2, dtype: object

In [365]:
train = combine[:train.shape[0]]
test = combine[train.shape[0]:]

In [366]:
train = train.drop(['pet_id','issue_date','listing_date'],axis=1)
test = test.drop(['pet_id','issue_date','listing_date'],axis=1)

In [367]:
train.head(3)

,condition,color_type,length(m),height(cm),X1,X2,issue_month,listing_month,days,serial,serial2
0,2.0,Brown Tabby,0.80,7.78,13,9,7,9,73,ANSL_6,ANSL_69
1,1.0,White,0.72,14.19,13,9,11,12,1862,ANSL_6,ANSL_66
2,1.0,Brown,0.15,40.90,15,4,9,10,752,ANSL_6,ANSL_69


## One Hot Encoding

In [368]:
train.select_dtypes(exclude='number').columns.to_list()

['color_type', 'serial', 'serial2']

In [369]:
train=pd.get_dummies(train)
test=pd.get_dummies(test)

In [370]:
remain = set(train.columns)-set(test.columns)

In [371]:
list(remain)

['color_type_Black Tiger', 'color_type_Brown Tiger']

In [372]:
train = train.drop(remain, axis=1)

In [373]:
train.head(3)

,condition,length(m),height(cm),X1,X2,issue_month,listing_month,days,color_type_Agouti,color_type_Apricot,...,serial2_ANSL_67,serial2_ANSL_68,serial2_ANSL_69,serial2_ANSL_70,serial2_ANSL_71,serial2_ANSL_72,serial2_ANSL_73,serial2_ANSL_74,serial2_ANSL_75,serial2_ANSL_76
0,2.0,0.80,7.78,13,9,7,9,73,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1.0,0.72,14.19,13,9,11,12,1862,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0.15,40.90,15,4,9,10,752,0,0,...,0,0,1,0,0,0,0,0,0,0


In [374]:
combine = pd.concat([train, test]).reset_index(drop=True)
combine.head(3)

,condition,length(m),height(cm),X1,X2,issue_month,listing_month,days,color_type_Agouti,color_type_Apricot,...,serial2_ANSL_67,serial2_ANSL_68,serial2_ANSL_69,serial2_ANSL_70,serial2_ANSL_71,serial2_ANSL_72,serial2_ANSL_73,serial2_ANSL_74,serial2_ANSL_75,serial2_ANSL_76
0,2.0,0.80,7.78,13,9,7,9,73,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1.0,0.72,14.19,13,9,11,12,1862,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0.15,40.90,15,4,9,10,752,0,0,...,0,0,1,0,0,0,0,0,0,0


## Preprocessing

In [375]:
from sklearn import preprocessing
# Get column names first
names = combine.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(combine)
combine = pd.DataFrame(scaled_df, columns=names)

In [376]:
train = combine[: train.shape[0]]
test = combine[train.shape[0]: ]

In [377]:
from sklearn.model_selection import train_test_split
x1_train,x1_test,y1_train,y1_test=train_test_split(train, y2, test_size=0.2,random_state=44,shuffle=True)

In [378]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

In [379]:
model1 = XGBClassifier()
model1.fit(x1_train, y1_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [380]:
train_out = model1.predict(train)
test_out = model1.predict(test)
val_out = model1.predict(x1_test)

In [381]:
train2 = pd.DataFrame(train, columns = names)
test2 = pd.DataFrame(test, columns = names)

In [382]:
train2.head(3)

,condition,length(m),height(cm),X1,X2,issue_month,listing_month,days,color_type_Agouti,color_type_Apricot,...,serial2_ANSL_67,serial2_ANSL_68,serial2_ANSL_69,serial2_ANSL_70,serial2_ANSL_71,serial2_ANSL_72,serial2_ANSL_73,serial2_ANSL_74,serial2_ANSL_75,serial2_ANSL_76
0,1.494263,1.024225,-1.514343,1.169789,1.262750,0.062139,0.622139,-0.712206,-0.014935,-0.024393,...,-0.196472,-0.196472,5.089794,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.183557
1,0.143981,0.747384,-1.020842,1.169789,1.262750,1.277405,1.457140,0.916017,-0.014935,-0.024393,...,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.183557
2,0.143981,-1.225103,1.035542,1.475018,-0.157894,0.669772,0.900472,-0.094228,-0.014935,-0.024393,...,-0.196472,-0.196472,5.089794,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.183557


In [383]:
test2.head(3)

,condition,length(m),height(cm),X1,X2,issue_month,listing_month,days,color_type_Agouti,color_type_Apricot,...,serial2_ANSL_67,serial2_ANSL_68,serial2_ANSL_69,serial2_ANSL_70,serial2_ANSL_71,serial2_ANSL_72,serial2_ANSL_73,serial2_ANSL_74,serial2_ANSL_75,serial2_ANSL_76
18834,-1.206301,1.266460,1.176432,-0.814202,0.694493,0.365955,0.622139,3.229570,-0.014935,-0.024393,...,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,5.089794,-0.183557
18835,0.143981,-1.536548,-1.596722,-0.814202,-1.010281,1.277405,-0.491196,-0.620283,-0.014935,-0.024393,...,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,5.447898
18836,0.143981,-0.913657,1.059409,-0.814202,0.694493,0.973589,-0.769530,1.040705,-0.014935,-0.024393,...,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.196472,-0.183557


In [384]:
train2['test_out'] = train_out
test2['test_out'] = test_out

In [385]:
x2_train,x2_test,y2_train,y2_test=train_test_split(train2,y1,test_size=0.2,random_state=44)

In [386]:
model2 = XGBClassifier()
model2.fit(x2_train, y2_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [387]:
test_out2 = model2.predict(test)
val_out2 = model2.predict(x2_test)

### Accuracy

In [388]:
s1=f1_score(y1_test,val_out,average='weighted')
s2=f1_score(y2_test,val_out2,average='weighted')
accuracy=100*((s1+s2)/2)
accuracy

89.41692228357765

In [389]:
sub_new=pd.DataFrame({
    "pet_id":test_id,
    "breed_category":test_out2,
    "pet_category":test_out
})
sub_new.to_csv("/kaggle/working/submission.csv",index=False)